# GROMACS Resume -  Start a long GROMACS run that times out, and resume it

On the supercomputers, a maximum runtime of 24h is enforced. If a longer GROMACS run is needed, the run can easily be resumed. One of the outputs provided by the Rush `gmx` module is designed to be used as the input to the `gmx_resume` module, which will resume the run from the latest checkpoint. The outputs to this module are identical to those of the `gmx` module itself, so the run can be resumed as many times as necessary to finish it.

## 0.0) Imports

In [ ]:
from pathlib import Path
import rush

In [ ]:
# |hide
# Users won't generally create a workspace
# We nuke to ensure run is reproducible
import os

WORK_DIR = Path.home() / "qdx" / "tutorial-gmx-resume"

if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=False)

os.makedirs(WORK_DIR, exist_ok=True)

2024-03-21 14:35:57,255 - rush - INFO - Restoring by default via env


In [ ]:
RUSH_TOKEN = os.getenv("RUSH_TOKEN") or "YOUR_TOKEN_HERE"
client = rush.build_blocking_provider_with_functions(access_token=RUSH_TOKEN)

2024-03-21 14:35:57,278 - rush - INFO - Restoring by default via env


In [ ]:
# |hide
# We hide this because users will generally not set a workspace, and won't restore by default
client = rush.build_blocking_provider_with_functions(
    batch_tags=["tutorial-resume-gmx"],
    workspace=WORK_DIR,
)

2024-03-21 14:35:57,952 - rush - INFO - Restoring by default via env


## 0.1) Input Download and Selection

In [ ]:
!pdb_fetch '1B39' | pdb_selchain -A | pdb_delhetatm > '1B39_A_nohet.pdb'

## 1) Input Preparation
We'll make sure to set `naming_scheme="charmm"` so that the names of the residues (i.e. amino acids) and atoms in the output will have the correct names for using with the CHARMM force field.

In [ ]:
_, prepared_protein_pdb = client.prepare_protein(Path.cwd() / "1B39_A_nohet.pdb", naming_scheme="charmm")

2024-03-21 14:36:00,622 - rush - INFO - Trying to restore job with tags: ['tutorial-resume-gmx'] and path: github:talo/prepare_protein/947cdbc000031e192153a20a9b4a8fbb12279102#prepare_protein_tengu
2024-03-21 14:36:00,694 - rush - WARNING - Multiple module instances found with the same tags and path
2024-03-21 14:36:00,695 - rush - WARNING - Multiple module instances found with the same tags and path
2024-03-21 14:36:00,695 - rush - WARNING - Multiple module instances found with the same tags and path
2024-03-21 14:36:00,695 - rush - WARNING - Multiple module instances found with the same tags and path
2024-03-21 14:36:00,696 - rush - WARNING - Multiple module instances found with the same tags and path
2024-03-21 14:36:00,762 - rush - INFO - Restoring job from previous run with id 3e4e10dd-f7cf-4e33-b759-cc73d6ae11c9


## 2.1) Run GROMACS (modules: gmx)
Next we will run a molecular dynamics simulation on our protein using gromacs via the `gmx` module.

We'll set "force_field" to "charmm36-ljpme" in our config dictionary, the latest official version of the CHARMM force field.

In [ ]:
resume_files_first, streaming_outputs, static_outputs, *rest = client.gmx(
    None,
    prepared_protein_pdb,
    None,
    {
        "params_overrides": {
            "nvt": {"nsteps": 2000},
            "npt": {"nsteps": 2000},
            "md": {"nsteps": 10000},
        },
        "force_field": "charmm36-ljpme",
        "num_gpus": 1,
    }
    resources={"gpus": 1, "storage": 1, "storage_units": "GB"},
)

2024-03-21 14:36:00,773 - rush - INFO - Trying to restore job with tags: ['tutorial-resume-gmx'] and path: github:talo/tengu-gmx/04cff2931b995c33263dfdb477d7f09c8bbd75a7#gmx_tengu
2024-03-21 14:36:00,822 - rush - INFO - Restoring job from previous run with id f6d416cf-0cde-4bf1-8ccd-44c0e71aa0d9


# Downloading Results
Let's download the logs and confirm that GROMACS ran using the requested force field.

streaming_outputs.download("streaming_outputs.tar.gz")

In [ ]:
import tarfile

with tarfile.open(client.workspace / "objects" / "streaming_outputs.tar.gz", "r") as tf:
    logs = [
        tf.extractfile(member).read()
        for member in tf
        if "log" in member.name and member.isfile()
    ]
    for log in enumerate(logs):
        for line in log.split("\n"):
            if "charmm" in line:
                print(line)

In [ ]:
with open(client.workspace / "objects" / "gmx_output_frame_0.pdb", "r") as f:
    print(str.join("", f.readlines()[0:10]))